In [1]:
# 作业3：感知机（上）
'''
    利用tanh(x)对符号函数sign(x)的近似：
    讨论近似后，公式（1）的凹凸性
'''

'\n    利用tanh(x)对符号函数sign(x)的近似：\n    1）讨论近似后，公式（1）的凹凸性；\n    2）分别利用随机梯度，最小批次随机梯度和梯度下降对（1）近似后进行优化，并观测优化过程与结果的关系。\n'

【感知机】
给定训练样本集$\begin{aligened}D = {(x_{i}, y_{i})}, i=1, ..., N\end{aligened}$。 一个自然的想法是让被正确
分类样本的数量最大化。被正确分类样本的数量可以被表示为以下损失：

$\begin{aligened}L(w, b) = \sum_{i=1}^{N}sign(g(x_{i};w,b)) y_{i}\end{aligened}$  （1）

其中，函数$\begin{aligened}g(x_{i};w,b)\end{aligened}$表示$\begin{aligened}x\end{aligened}$为输入，而$\begin{aligened}w\end{aligened}$和$\begin{aligened}b\end{aligened}$是参数。当$\begin{aligened}x_{i}\end{aligened}$被正确分类，那么
$\begin{aligened}sign(g(x_{i};w,b)) y_{i}=1\end{aligened}$；当x被错误分类，那么$\begin{aligened}sign(g(x_{i};w,b)) y_{i}=-1\end{aligened}$。最
大化损失函数就可以找到最优参数$\begin{aligened}w\end{aligened}$和$\begin{aligened}b\end{aligened}$。$\begin{aligened}y=sign(x)\end{aligened}$是不可导函数，我们拟
利用$\begin{aligened}tanh(x)\end{aligened}$函数对$\begin{aligened}sign(x)\end{aligened}$进行近似：

$\begin{aligened}y=tanh(kx)\end{aligened}$ （2）

其中， $\begin{aligened}k(k > 0)\end{aligened}$参数控制函数$\begin{aligened}tanh(kx)\end{aligened}$的形状， $\begin{aligened}k\end{aligened}$越大$\begin{aligened}tanh(kx)\end{aligened}$就越接近$\begin{aligened}sign(x)\end{aligened}$。

In [2]:
# 导库
import sympy as sp
import numpy as np

In [3]:
# 定义tanh函数和sign函数（不允许用np.tanh或np.sign）
# tanh函数
def tanh(x):
    num = np.exp(x) - np.exp(-x)
    den = np.exp(x) + np.exp(-x)
    return num / den

# sign函数
def sign(x):
    if x > 0:
        return 1
    elif x == 0:
        return 0
    elif x < 0:
        return -1

In [4]:
# 损失函数
def loss_function(x, y, w, b, k=1, func='tanh'):
    g_value = np.dot(w.T, x) + b
    func_result = np.zeros(len(g_value))
    # 判断函数类型（sign还是tanh）
    if func == 'sign':
        func_result = sign(g_value)
    elif func == 'tanh':
        func_result = tanh(k * g_value)
    classifier = np.dot(func_result.T, y)
    loss = np.sum(classifier)
    return loss

In [5]:
# 计算梯度
def gradient(x, y, w, b, k):
    gradient_w = np.zeros_like(w)
    gradient_b = np.zeros_like(b)
    for i in range(200):
        prediction = tanh(np.dot(w, k * x[i]) + b)
        judge = prediction * y[i]
        if judge <= 0:
            gradient_w += x[i] * (1 / np.cosh(np.dot(w, x[i]) + b) ** 2) * y[i]
            gradient_b += (1 / np.cosh(np.dot(w, x[i]) + b) ** 2) * y[i]
    return gradient_w, gradient_b

In [6]:
# 函数凹凸性判断
# 计算损失函数的海森矩阵并打印其特征值（关于x和y的函数）
# 通过约分法、代值法等判断其正定性以判断函数凹凸性（若海森矩阵半正定，则函数为凸函数；反之凹函数）
def judge_convexity_and_concavity(w_value, b_value, k_value):
    x, y, w, b, k = sp.symbols('x y w b k')
    func = sp.tanh(k * (w * x ++ b)) * y
    first_derivative_x = sp.diff(func, x)
    first_derivative_y = sp.diff(func, y)
    second_derivate_x_x = sp.diff(first_derivative_x, x).evalf(subs ={'w': w_value, 'k': k_value})
    second_derivate_x_y = sp.diff(first_derivative_x, y).evalf(subs ={'w': w_value, 'k': k_value})
    second_derivate_y_y = sp.diff(first_derivative_y, y).evalf(subs ={'w': w_value, 'k': k_value})
    second_derivate_y_x = sp.diff(first_derivative_y, x).evalf(subs ={'w': w_value, 'k': k_value})
    hessian_matrix = sp.Matrix([[second_derivate_x_x, second_derivate_x_y],
                               [second_derivate_y_x, second_derivate_y_y]])
    eigenvalues = hessian_matrix.eigenvals()
    print('When w value is {0}, k value is {1}, \n'
          'the eigenvalue of the Hessian matrix of the function is :\n'
          '{2}\n'.format(w_value, k_value, eigenvalues.keys()))

In [10]:
# 判断凹凸性
judge_convexity_and_concavity(w_value=10, b_value=1, k_value=10)
judge_convexity_and_concavity(w_value=10, b_value=1, k_value=100)

When w value is 10, k value is 10, 
the eigenvalue of the Hessian matrix of the function is :
dict_keys([-10000.0*y*tanh(10*b + 100*x)/cosh(10*b + 100*x)**2 - 10000.0*sqrt(y**2*tanh(10*b + 100*x)**2 + 0.0001)/cosh(10*b + 100*x)**2, -10000.0*y*tanh(10*b + 100*x)/cosh(10*b + 100*x)**2 + 10000.0*sqrt(y**2*tanh(10*b + 100*x)**2 + 0.0001)/cosh(10*b + 100*x)**2])

When w value is 10, k value is 100, 
the eigenvalue of the Hessian matrix of the function is :
dict_keys([-1000000.0*y*tanh(100*b + 1000*x)/cosh(100*b + 1000*x)**2 - 1000000.0*sqrt(y**2*tanh(100*b + 1000*x)**2 + 1.0e-6)/cosh(100*b + 1000*x)**2, -1000000.0*y*tanh(100*b + 1000*x)/cosh(100*b + 1000*x)**2 + 1000000.0*sqrt(y**2*tanh(100*b + 1000*x)**2 + 1.0e-6)/cosh(100*b + 1000*x)**2])

